In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [16]:
base_path = 'D:\\KE Notes\\Developing intelligent systems for BA\\Day3\\2a\\'
fin_data =  pd.read_csv(base_path+'FinancialData.csv')
sgd_preds = pd.read_csv(base_path+'pred_l_sg_xch.csv')
pound_preds = pd.read_csv(base_path+'pred_l_pound_xch.csv')

In [17]:
fin_data['SG/P'] = fin_data['SG/D']/fin_data['P/D']

In [18]:
# sg_value , us_value, pound_value (from previous week)
# prev_sg_value = sg_value
# prev_us_value = us_value
# prev_pound_value = pound_value
# sg_value = prev_sg_value + us_sg_qty*prev_us_value/((SG/D)*1.01) - sg_us_qty*prev_sg_value
#                          + pound_sg_qty*prev_pound_value/((SG/P)*1.01) - sg_pound_qty*prev_sg_value
# us_value = prev_us_value + sg_us_qty*prev_sg_value*((SG/D)/1.01) - us_sg_qty*prev_us_value
#                          + pound_us_qty*prev_pound_value*((P/D)/1.01) - us_pound_qty*prev_us_value
# pound_value = prev_pound_value + us_pound_qty*prev_us_value/((P/D)*1.01) - pound_us_qty*prev_pound_value
#                                + sg_pound_qty*prev_sg_value*((SG/P)/1.01) - pound_sg_qty*prev_pound_value
# sg_value_in_us = sg_value * SG/D
# pound_value_in_us = pound_value * P/D
# y = sg_value_in_us          + us_value          + pound_value_in_us + 
#     sg_value_in_us*interest + us_value*interest + pound_value_in_us*interest

In [19]:
def estimate_total(x,prev_val,qty_val,sgd_preds,pound_preds,wdw):
    #prev_val --> [sg_value, us_value, pound_value]
    #qty_val --> [sg_us_qty,pound_us_qty,sg_pound_qty]
    #pred_xch_val  --> [SG/D,P/D]
    prev_sg_val = prev_val[0]
    prev_us_val = prev_val[1]
    prev_pound_val = prev_val[2]

    for w in range(1,wdw+1):
#         print(w)
        pred_xch_val = [sgd_preds.iloc[x+w-1: x+w,x].values[0],
                        pound_preds.iloc[x+w-1: x+w,x].values[0],
                        sgd_preds.iloc[x+w-1: x+w,x].values[0]/pound_preds.iloc[x+w-1: x+w,x].values[0]] 
        if w == 1:
            prev_sg_val = prev_val[0]
            prev_us_val = prev_val[1]
            prev_pound_val = prev_val[2]
        else:
            prev_sg_val = sg_val
            prev_us_val = us_val
            prev_pound_val = pound_val          
        qty_sg_us_val = qty_val[0+3*(w-1)]
        qty_pound_us_val = qty_val[1+3*(w-1)]
        qty_sg_pound_val = qty_val[2+3*(w-1)]
        # reductions
        sg_val = prev_sg_val - max(qty_sg_us_val,0)*prev_sg_val - max(qty_sg_pound_val,0)*prev_sg_val
        us_val = prev_us_val - abs(min(qty_sg_us_val,0))*prev_us_val - abs(min(qty_pound_us_val,0))*prev_us_val
        pound_val = prev_pound_val - max(qty_pound_us_val,0)*prev_pound_val - abs(min(qty_sg_pound_val,0))*prev_pound_val
        #print(sg_val,us_val,pound_val)
        if sg_val >= 0 and us_val >= 0 and pound_val >= 0:
            # additions
            sg_val = sg_val + abs(min(qty_sg_us_val,0))*prev_us_val/(pred_xch_val[0]*1.01) \
                            + abs(min(qty_sg_pound_val,0))*prev_pound_val/(pred_xch_val[2]*1.01) 
            us_val = us_val + max(qty_sg_us_val,0)*prev_sg_val*(pred_xch_val[0]/1.01) \
                            + max(qty_pound_us_val,0)*prev_pound_val*(pred_xch_val[1]/1.01) 
            pound_val = pound_val + abs(min(qty_pound_us_val,0))*prev_us_val/(pred_xch_val[1]*1.01) \
                                  + max(qty_sg_pound_val,0)*prev_sg_val*(pred_xch_val[2]/1.01) 
#             print(sg_val,us_val,pound_val)
        else:
            return 0
    sg_val_us = sg_val * pred_xch_val[0]
    pound_val_us = pound_val * pred_xch_val[1]
    #print(sg_val_us, us_val, pound_val_us)
    return sg_val_us + us_val + pound_val_us 

In [20]:
def cal_pop_fitness(x,prev_val, sgd_preds,pound_preds, pop, wdw):
     # Calculating the fitness value of each solution in the current population.
     # The fitness function calculates the sum of products between each input and its corresponding weight.
    fitness = []
    for p in pop:
        fitness.append(estimate_total(x,prev_val,p,sgd_preds,pound_preds,wdw))
    #fitness = np.sum(pop*equation_inputs, axis=1)
    return fitness
# cal_pop_fitness(prev_val, pred_xch_val,new_population)

In [21]:
def select_mating_pool(pop, fitness, num_parents,best_ind=False):
    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
    parents = np.empty((num_parents, pop.shape[1]))
    for parent_num in range(num_parents):
        max_fitness_idx = np.where(fitness == np.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num, :] = pop[max_fitness_idx, :]
        max_fitness = fitness[max_fitness_idx]
        fitness[max_fitness_idx] = -99999999999
    if best_ind:
        return parents[0],max_fitness
    else:
        return parents

In [22]:
def crossover(parents, offspring_size):
    offspring = np.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually, it is at the center.
    crossover_point = offspring_size[1]//2
    
    for k in range(offspring_size[0]):
        # Index of the first parent to mate.
        parent1_idx = k%parents.shape[0]
        # Index of the second parent to mate.
        parent2_idx = (k+1)%parents.shape[0]
        # The new offspring will have its first half of its genes taken from the first parent.
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        # The new offspring will have its second half of its genes taken from the second parent.
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring

In [23]:
def mutation(offspring_crossover,mutation_rate):
    # Mutation changes a single gene in each offspring randomly.
    mutated_offspring = []
    for idx in range(offspring_crossover.shape[0]):
        for swapped in range(len(offspring_crossover[idx])):
            if np.random.random() < mutation_rate:
                swap_with = int(np.random.random() * len(offspring_crossover[idx]))
                x = offspring_crossover[idx][swapped]
                y = offspring_crossover[idx][swap_with]
                
                offspring_crossover[idx][swapped] = y
                offspring_crossover[idx][swap_with] = x
        mutated_offspring.append(offspring_crossover[idx])
        # The random value to be added to the gene.
        #random_value = np.random.uniform(-1.0, 1.0, 1)
        #random_byte = np.random.randint(0,3,1)[0]
        #offspring_crossover[idx, random_byte] = offspring_crossover[idx, random_byte] + random_value
    return mutated_offspring

In [24]:
def actual_total(x,prev_val,qty_val,act_xch_val):
    #prev_val --> [sg_value, us_value, pound_value]
    #qty_val --> [sg_us_qty,pound_us_qty,sg_pound_qty]
    #act_xch_val  --> [SG/D,P/D,(SG/D)/(P/D)]
    prev_sg_val = prev_val[0]
    prev_us_val = prev_val[1]
    prev_pound_val = prev_val[2]
    # reductions
    sg_val = prev_sg_val - max(qty_val[0],0)*prev_sg_val - max(qty_val[2],0)*prev_sg_val
    us_val = prev_us_val - abs(min(qty_val[0],0))*prev_us_val - abs(min(qty_val[1],0))*prev_us_val
    pound_val = prev_pound_val - max(qty_val[1],0)*prev_pound_val - abs(min(qty_val[2],0))*prev_pound_val
    #print(sg_val,us_val,pound_val)
    if sg_val >= 0 and us_val >= 0 and pound_val >= 0:
        # additions
        sg_val = sg_val + abs(min(qty_val[0],0))*prev_us_val/(act_xch_val[0]*1.01) \
                        + abs(min(qty_val[2],0))*prev_pound_val/(act_xch_val[2]*1.01) 
        us_val = us_val + max(qty_val[0],0)*prev_sg_val*(act_xch_val[0]/1.01) \
                        + max(qty_val[1],0)*prev_pound_val*(act_xch_val[1]/1.01) 
        pound_val = pound_val + abs(min(qty_val[1],0))*prev_us_val/(act_xch_val[1]*1.01) \
                              + max(qty_val[2],0)*prev_sg_val*(act_xch_val[2]/1.01) 
        #print(sg_val,us_val,pound_val)
        sg_val_us = sg_val * act_xch_val[0]
        pound_val_us = pound_val * act_xch_val[1]
        #print(sg_val_us, us_val, pound_val_us)
        return sg_val_us + us_val + pound_val_us , [sg_val, us_val, pound_val]
    else:
        return 0,0,0,0

In [25]:
def ga_part(x,wdw,sol_per_pop,num_generations,mutation_rate,prev_val):
    
    num_genes = wdw*3
    pop_size = (sol_per_pop,num_genes)
    new_population = np.around(np.random.uniform(low=-1.0, high=1.0, size=pop_size),decimals=3)
    num_parents_mating = int(sol_per_pop/2)
    
    for generation in range(num_generations):
        #print("Generation ", generation)
        # Measuring the fitness of each chromosome in the population.
        fitness = cal_pop_fitness(x,prev_val, sgd_preds,pound_preds,new_population, wdw)
        # Selecting the best parents in the population for mating.
        parents = select_mating_pool(new_population, fitness, num_parents_mating)
        # Generating next generation using crossover.
        offspring_crossover = crossover(parents,offspring_size=(pop_size[0]-parents.shape[0], num_genes))
        # Adding some variations to the offsrping using mutation.
        offspring_mutation = mutation(offspring_crossover,mutation_rate)
        # Creating the new population based on the parents and offspring.
        new_population[0:parents.shape[0], :] = parents
        new_population[parents.shape[0]:, :] = offspring_mutation
    fitness = cal_pop_fitness(x,prev_val, sgd_preds,pound_preds,new_population, wdw)
    
    best_ind,max_fitness = select_mating_pool(new_population, fitness, 1,best_ind=True)
    #print('best_individual:',best_ind[:3],'with fitness', max_fitness)
    return best_ind

In [26]:
#x = 0
init_bal = [10000,10000,10000]
for x in range(103):
    act_xch_val = [fin_data.loc[501+x,'SG/D'],
                   fin_data.loc[501+x,'P/D'],
                   fin_data.loc[501+x,'SG/D']/fin_data.loc[501+x,'P/D']]
    best_ind = ga_part(x, wdw = 1, sol_per_pop = 50, num_generations = 50, mutation_rate = 0.2,
                       prev_val = init_bal)
    tot_bal,curr_bal = actual_total(x, prev_val = init_bal, qty_val = best_ind, act_xch_val = act_xch_val)
    print('week',502+x,' init_bal',init_bal,'curr_bal',np.around(curr_bal,decimals=4),round(tot_bal,4))
    init_bal = np.around(curr_bal,decimals=4)


week 502  init_bal [10000, 10000, 10000] curr_bal [ 9720.     10311.0495  9924.1035] 32435.9059
week 503  init_bal [ 9720.     10311.0495  9924.1035] curr_bal [10184.367  10042.9622  9881.0297] 32539.2476
week 504  init_bal [10184.367  10042.9622  9881.0297] curr_bal [10537.3944  9842.103   9846.7626] 32478.8231
week 505  init_bal [10537.3944  9842.103   9846.7626] curr_bal [10305.5717 10090.929   9790.4039] 32685.5529
week 506  init_bal [10305.5717 10090.929   9790.4039] curr_bal [10907.3277  9747.8374  9733.3695] 32574.7372
week 507  init_bal [10907.3277  9747.8374  9733.3695] curr_bal [11534.1053  9396.9153  9670.5751] 32473.1799
week 508  init_bal [11534.1053  9396.9153  9670.5751] curr_bal [10426.8312  9219.8241 10279.321 ] 32532.5435
week 509  init_bal [10426.8312  9219.8241 10279.321 ] curr_bal [10260.0019  9404.9291 10234.6205] 32586.8954
week 510  init_bal [10260.0019  9404.9291 10234.6205] curr_bal [ 9808.5618  9906.6495 10111.6645] 32431.861
week 511  init_bal [ 9808.5618  9

week 578  init_bal [ 5071.4354  9414.1914 12241.4839] curr_bal [ 4970.0067  9648.5195 12137.8806] 33078.6707
week 579  init_bal [ 4970.0067  9648.5195 12137.8806] curr_bal [ 4947.6123  9378.361  12304.0788] 33159.5174
week 580  init_bal [ 4947.6123  9378.361  12304.0788] curr_bal [ 5898.9346  9003.2266 12170.5215] 32734.407
week 581  init_bal [ 5898.9346  9003.2266 12170.5215] curr_bal [ 5651.1793  8968.104  12279.4636] 32603.5803
week 582  init_bal [ 5651.1793  8968.104  12279.4636] curr_bal [ 6040.3181  8824.6143 12223.789 ] 32848.497
week 583  init_bal [ 6040.3181  8824.6143 12223.789 ] curr_bal [ 6016.1568  8871.6824 12203.6482] 32593.4685
week 584  init_bal [ 6016.1568  8871.6824 12203.6482] curr_bal [ 5582.9935  8374.6094 12658.135 ] 32178.644
week 585  init_bal [ 5582.9935  8374.6094 12658.135 ] curr_bal [ 5359.6738  8304.2065 12778.5756] 32209.7798
week 586  init_bal [ 5359.6738  8304.2065 12778.5756] curr_bal [ 5415.1378  8055.0803 12909.8413] 32201.8665
week 587  init_bal [ 5

In [ ]:
using exponential prediction
window - 1 
week 605 curr_bal [1.12100000e-01 1.29544001e+04 1.20986440e+04] 32787.7741
using polynomial prediction
window - 1 
week 605 curr_bal [ 3339.8482  7043.1644 14253.112 ] 32297.173